In [27]:
import rpy2.robjects as ro
from rpy2.robjects import r
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

import pandas as pd
import re
import json

data_dir = "./DATA/data.json"
with open(data_dir, "r") as f:
    json_data = json.load(f)

json_data

{'data': [[{'formulation': [{'excipients': 'Acetic Acid',
      'kind': 'General',
      'max': {'value': '4.4 %w/v', 'unit': 'Acidifier'},
      'use range': {'min': '-', 'max': '0 ~ 4.4'},
      'input range': {'min': '1', 'max': '12'}},
     {'excipients': 'Hydrochloric Acid',
      'kind': 'General',
      'max': {'value': '78.09 mg', 'unit': 'Acidifier'},
      'use range': {'min': '-', 'max': '0 ~ 78.09'},
      'input range': {'min': '2', 'max': '24'}}],
    'CQAs': ['appearance']}]]}

In [37]:
def header(value):
    excipients_header = []
    cqa_header = []
    header = []
    formulation = value['formulation']
    cqas = value['CQAs']
    
    for i in range(len(formulation)):
        export_dict = formulation[i]
        excipients = export_dict.get("excipients")
        excipients_header.append(excipients)
    for j in range(len(cqas)):
        cqa = cqas[j]
        cqa_header.append(cqa)
    header = excipients_header + cqa_header
        
    return (excipients_header, header)
    
def DataProcessing(value):
    print("## Data 전처리 ##")
    origin_data = value['data'][0][0]
    input = origin_data['formulation']
    
    ## 1. formulation_excipients 공백 -> . 로 변경
    for i in range(len(input)):
        origin_excipients = input[i]['excipients']
        blank_remove_excipients = origin_excipients.replace(" ", ".")
        sign_remove_excipients = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", ".", blank_remove_excipients)
        input[i]['excipients'] = sign_remove_excipients
    
    excipients = [item['excipients'] for item in input]
    input_range_min = [item['input range']['min'] for item in input]
    input_range_max = [item['input range']['max'] for item in input]
    
    processing_Data = pd.DataFrame({
        'excipients': excipients,
        'input.range.min': input_range_min,
        'input.range.max': input_range_max
    })
    
    return processing_Data

def runScript(value):
    try:
        input_df = DataProcessing(value)
        with localconverter(ro.default_converter + pandas2ri.converter):
            pandas2ri.activate()
            
            input_data_r = ro.conversion.py2rpy(input_df)
            
            ro.r.assign("data", input_data_r)
            r = ro.r
            r.source("./Rscript/CCDDoE.R")
            test_df = ro.conversion.rpy2py(ro.globalenv['test_df'])
        return test_df
    except Exception as e:
        print(e)

# Data PreProcessing

In [31]:
import json
import pandas as pd
import re
import csv


origin_data = json_data['data'][0][0]
origin_data

input = origin_data['formulation']
input

## 1. formulation_excipients 공백 -> . 로 변경
for i in range(len(input)):
        origin_excipients = input[i]['excipients']
        blank_remove_excipients = origin_excipients.replace(" ", ".")
        sign_remove_excipients = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", ".", blank_remove_excipients)
        input[i]['excipients'] = sign_remove_excipients

excipients = [item['excipients'] for item in input]
input_range_min = [item['input range']['min'] for item in input]
input_range_max = [item['input range']['max'] for item in input]

input_df = pd.DataFrame({
    'excipients': excipients,
    'input.range.min': input_range_min,
    'input.range.max': input_range_max
})

input_df


,excipients,input.range.min,input.range.max
0,Acetic.Acid,1,12
1,Hydrochloric.Acid,2,24


# Python -> R

In [21]:
CCD_DOE_Script = "./Deploy_RPY2_5/Rscript/CCDDoE.R"


with localconverter(ro.default_converter + pandas2ri.converter):
    pandas2ri.activate()

    input_data_r = ro.conversion.py2rpy(input_df)
    ro.r.assign("data", input_data_r)
    
    r = ro.r
    print(r.source(CCD_DOE_Script))
    
    #############
    ## CCD_DOE ##
    #############
    CCD_DF = ro.conversion.rpy2py(ro.globalenv['df.ccd_DoE'])
    

[1] "tools"     "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"     
[1] "tools"     "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"     
[1] "##LIBRARY LOAD _ START##"


Exception ignored from cffi callback <function _consolewrite_ex at 0x000002A99BAE8550>:
Traceback (most recent call last):
  File "c:\Users\DCU\anaconda3\envs\pharmaienv\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 133, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "c:\Users\DCU\anaconda3\envs\pharmaienv\lib\site-packages\rpy2\rinterface_lib\conversion.py", line 138, in _cchar_to_str_with_maxlen
    s = ffi.string(c, maxlen).decode(encoding)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc7 in position 0: invalid continuation byte
Exception ignored from cffi callback <function _consolewrite_ex at 0x000002A99BAE8550>:
Traceback (most recent call last):
  File "c:\Users\DCU\anaconda3\envs\pharmaienv\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 133, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "c:\Users\DCU\anaconda3\envs\pharmaienv\lib\site-packages\rpy2

[1] "##LIBRARY LOAD _ END##"
[1] "create list success"
[1] "df start"


R[write to console]: creating full factorial with 4 runs ...




Time difference of 5.061607 secs
o{'value': [5.06160688], 'visible': [1] FALSE
, }


In [41]:
CCD_DF

,Block.ccd,Hydrochloric.Acid,Acetic.Acid
C1.1,1,2.0,1.0
C1.2,1,24.0,1.0
C1.3,1,2.0,12.0
C1.4,1,24.0,12.0
C1.5,1,13.0,6.5
C1.6,1,13.0,6.5
C1.7,1,13.0,6.5
C1.8,1,13.0,6.5
C1.9,1,13.0,6.5
S2.1,2,2.0,6.5


In [44]:
CCD_DF.iloc[0]

Block.ccd              1
Hydrochloric.Acid    2.0
Acetic.Acid          1.0
Name: C1.1, dtype: object

In [47]:
excipients_header, header_result = header(origin_data)

experiment_dict = {}
experiment_list = []
experiment_result = []

for i in range(len(CCD_DF)):
    for j in range(len(excipients_header)):
        excipients = excipients_header[j]
        
        blank_remove_excipients = excipients.replace(" ", ".")
        sign_remove_excipients = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", ".", blank_remove_excipients)
        experiment_data = CCD_DF.iloc[i][sign_remove_excipients]
        experiment_dict[excipients] = experiment_data
    experiment_list.append(experiment_dict.copy())

res_dict = {}
res_dict["header"] = header_result
res_dict["experiment_data"] = experiment_list
code = "000"
msg = "success"
res_dict


{'header': ['Acetic.Acid', 'Hydrochloric.Acid', 'appearance'],
 'experiment_data': [{'Acetic.Acid': 1.0, 'Hydrochloric.Acid': 2.0},
  {'Acetic.Acid': 1.0, 'Hydrochloric.Acid': 24.0},
  {'Acetic.Acid': 12.0, 'Hydrochloric.Acid': 2.0},
  {'Acetic.Acid': 12.0, 'Hydrochloric.Acid': 24.0},
  {'Acetic.Acid': 6.5, 'Hydrochloric.Acid': 13.0},
  {'Acetic.Acid': 6.5, 'Hydrochloric.Acid': 13.0},
  {'Acetic.Acid': 6.5, 'Hydrochloric.Acid': 13.0},
  {'Acetic.Acid': 6.5, 'Hydrochloric.Acid': 13.0},
  {'Acetic.Acid': 6.5, 'Hydrochloric.Acid': 13.0},
  {'Acetic.Acid': 6.5, 'Hydrochloric.Acid': 2.0},
  {'Acetic.Acid': 6.5, 'Hydrochloric.Acid': 24.0},
  {'Acetic.Acid': 1.0, 'Hydrochloric.Acid': 13.0},
  {'Acetic.Acid': 12.0, 'Hydrochloric.Acid': 13.0},
  {'Acetic.Acid': 6.5, 'Hydrochloric.Acid': 13.0}]}